This script is used to perform lesion segmentation in fundus photographs. The lesions that can be segmented are hard exudates, soft exudates, microaneurysms and hemorrhages. The segmentation is based on a UNet, a CNN that takes an image as an input and that outputs a probability map indicating for every pixel the probability of belonging to a certain type of lesion or not.

In [ ]:
!pip install tensorlayer==1.11.1
%tensorflow_version 1.x

In [ ]:
# import necessary libraries

import numpy as np
import tensorflow as tf
import tensorlayer as tl
import tensorlayer.layers as tll
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import time

In [ ]:
# read in the train and test data

# Basepath depends on the lesion
# LesionType = 'SoftExudates'
LesionType = 'HardExudates'
# LesionType = 'Microaneurysms'
# LesionType = 'Hemorrhages'

In [ ]:
# read in train and test data in case Google DRIVE is used
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# basepath for google drive
# Basepath = '/content/drive/My Drive/Stage_ENT_Studios/Data/IDRiD/' + LesionType + '/Arrays/'

# basepath for jupyter notebooks
Basepath = 'C:/Users/lunam/Documents/1steMaster/Stage/Data_FinalArrays/IDRiD/'+ LesionType+'/Arrays/'


# train data
train_images = np.float32(np.load(Basepath + 'train_images_Final.npy'))
print('Shape train images: {}'.format(train_images.shape))

train_annotations =  np.int32(np.load(Basepath + 'train_annotations_Final.npy'))
# train_annotations = np.expand_dims(train_annotations, axis = 3)
print('Shape train annotations: {}'.format(train_annotations.shape))

# test data
test_images = np.float32(np.load(Basepath + 'test_images_Final.npy'))
print('Shape test images: {}'.format(test_images.shape))

test_annotations = np.int32(np.load(Basepath + 'test_annotations_Final.npy'))
# test_annotations = np.expand_dims(test_annotations, axis = 3)
print('Shape test annotations: {}'.format(test_annotations.shape))

Shape train images: (324, 512, 512, 3)
Shape train annotations: (324, 512, 512)
Shape test images: (156, 512, 512, 3)
Shape test annotations: (156, 512, 512)


In [ ]:
# Load the tensorboard extension. Tensorboard will be used to visualize the training and testing of the network
%load_ext tensorboard

# basepath google drive
# base_path = '/content/drive/My Drive/Stage_ENT_Studios/Unet/Logs/'

# basepath jupyter notebooks
base_path = 'C:/Users/lunam/Documents/1steMaster/Stage/Code_Final/DR_classification/FeatureBasedClassification/UNet_Softmax/Logs/'+LesionType+'/'


# direction where the tensorboard files will be stored
log_dir_tens = base_path + 'Tensorboard_Logs/'
# direction where the trained models will be stored
log_dir_model = base_path + 'Trained_Model/'

In [ ]:
class BatchData():
    '''
    This class is used to create batches of images with their corresponding annotations
    These batches can then be fed to the Unet
    The numpy arrays of preprocessed images and corresponding annotations are given as an input to the class
    '''
    
    # resetting all values in case a new batch dataset is created
    images = []
    annotations = []
    batch_offset = 0
    epochs_completed = 0 


    def __init__(self, image_arrays, annotation_arrays):
        
        print("Initializing Batch Dataset Reader...")
        
        self.images = image_arrays
        self.annotations = annotation_arrays

        # resize the annotations to indicate that there is only one channel in this case
        self.annotations = np.expand_dims(self.annotations, 3)
        # this makes a binary mask of the annotations, there where annotations has a pixel value above 0, the pixel value will be set to 1
        # there where the pixel value is 0 or smaller, the pixel value is set to 0
        self.annotations = np.where(self.annotations>0, 1, 0)
        
        print ('Shape images: {}'.format(self.images.shape))
        print ('Shape annotations: {}'.format(self.annotations.shape))
    
    def next_batch(self, batch_size):
        '''
        This function can be used to everytime find the next batch of images and corresponding annotations
        The size of the batches is defined by the batch_size
        '''
        
        # start image of the batch
        start = self.batch_offset
        # set the starting value for the next batch
        self.batch_offset += batch_size 
        
        # all data has already been used
        if self.batch_offset > self.images.shape[0]:
            
            # images and annotations get shuffled randomly for the next epoch
            perm = np.arange(self.images.shape[0])
            np.random.shuffle(perm)
            self.images = self.images[perm]
            self.annotations = self.annotations[perm]
            
            # Start next epoch
            start = 0
            self.batch_offset = batch_size
        
        # end image of the batch
        end = self.batch_offset
        
        # return the images and corresponding annotations in this batch
        return self.images[start:end], self.annotations[start:end]

In [ ]:
def LoadBatchData(Train_Images, Train_Annotations, Test_Images, Test_Annotations):
    '''
    This function creates a batch dataset for the train and test set
    '''
    
    # create a batchdataset for the train set
    train_batch_data = BatchData(Train_Images, Train_Annotations)
            
    # create a batchdataset for the test set
    test_batch_data = BatchData(Test_Images, Test_Annotations)
            
    # return the train and test batch dataset
    return train_batch_data, test_batch_data 

In [ ]:
# The UNet network
def UNet(image, drop_prob = 0.1, init_filters = 64, reuse = False, is_train = True):
    '''This function defines the original UNet network'''
    
    # initialization of the weights
    W_init = tf.contrib.layers.xavier_initializer()
        
    # Unet network
                
    # LEFT part
    # input layer
    input_image = tl.layers.InputLayer(image)
            
    # Convolutional block 1
    conv2d_1 = tl.layers.Conv2d(input_image, init_filters, (3, 3), act= tf.nn.relu, W_init= W_init)
    conv2d_2 = tl.layers.Conv2d(conv2d_1, init_filters, (3, 3), act= tf.nn.relu, W_init= W_init)
    pool_1 = tl.layers.MaxPool2d(conv2d_2, (2, 2), (2, 2), name= 'maxpool_1')
    dropout_1 = tl.layers.DropoutLayer(pool_1, keep= 1-drop_prob, is_fix = True, is_train=is_train)
                
    # Convolutional block 2
    conv2d_3 = tl.layers.Conv2d(dropout_1, 2*init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
    conv2d_4 = tl.layers.Conv2d(conv2d_3, 2*init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
    pool_2 = tl.layers.MaxPool2d(conv2d_4, (2,2), (2, 2), name='maxpool_2')
    dropout_2 = tl.layers.DropoutLayer(pool_2, keep= 1-drop_prob, is_fix = True, is_train=is_train)
                
    # Convolutional block 3
    conv2d_5 = tl.layers.Conv2d(dropout_2, 4*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_6 = tl.layers.Conv2d(conv2d_5, 4*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    pool_3 = tl.layers.MaxPool2d(conv2d_6, (2,2), (2, 2), name='maxpool_3')
    dropout_3 = tl.layers.DropoutLayer(pool_3, keep= 1-drop_prob, is_fix = True, is_train=is_train)
                
    # Convolutional block 4
    conv2d_7 = tl.layers.Conv2d(dropout_3, 8*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_8 = tl.layers.Conv2d(conv2d_7, 8*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    pool_4 = tl.layers.MaxPool2d(conv2d_8, (2,2), (2, 2), name='maxpool_4')
    dropout_4 = tl.layers.DropoutLayer(pool_4, keep= 1-drop_prob, is_fix = True, is_train=is_train)
                
    # MIDDLE part
    conv2d_9 = tl.layers.Conv2d(dropout_4, 16*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_10 = tl.layers.Conv2d(conv2d_9, 16*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
                
                
    # RIGHT part
    # Convolutional block 1
    upsampling_1 = tl.layers.UpSampling2dLayer(conv2d_10, (2,2))
    concat_1 = tl.layers.ConcatLayer([upsampling_1, conv2d_8], 3)
    dropout_5 = tl.layers.DropoutLayer(concat_1, keep= 1-drop_prob, is_fix = True, is_train=is_train)
    conv2d_11 = tl.layers.Conv2d(dropout_5, 8*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_12 = tl.layers.Conv2d(conv2d_11, 8*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
                
    # Convolutional block 2
    upsampling_2 = tl.layers.UpSampling2dLayer(conv2d_12, (2,2))
    concat_2 = tl.layers.ConcatLayer([upsampling_2, conv2d_6], 3)
    dropout_6 = tl.layers.DropoutLayer(concat_2, keep= 1-drop_prob, is_fix = True, is_train=is_train)
    conv2d_13 = tl.layers.Conv2d(dropout_6, 4*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
    conv2d_14 = tl.layers.Conv2d(conv2d_13, 4*init_filters, (3,3), act= tf.nn.relu, W_init=W_init)
            
                
    # Convolutional block 3
    upsampling_3 = tl.layers.UpSampling2dLayer(conv2d_14, (2,2))
    concat_3 = tl.layers.ConcatLayer([upsampling_3,conv2d_4], 3)
    dropout_7 = tl.layers.DropoutLayer(concat_3, keep= 1-drop_prob, is_fix = True, is_train=is_train)
    conv2d_15 = tl.layers.Conv2d(dropout_7, 2*init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
    conv2d_16 = tl.layers.Conv2d(conv2d_15, 2*init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
                
    # Convolutional block 4
    upsampling_4 = tl.layers.UpSampling2dLayer(conv2d_16, (2,2))
    concat_4 = tl.layers.ConcatLayer([upsampling_4,conv2d_2], 3)
    dropout_8 = tl.layers.DropoutLayer(concat_4, keep= 1-drop_prob, is_fix = True, is_train=is_train)
    conv2d_17 = tl.layers.Conv2d(dropout_8, init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
    conv2d_18 = tl.layers.Conv2d(conv2d_17, init_filters, (3,3), act= tf.nn.relu, W_init= W_init)
                
                
    # ouput layer
    output_image = tl.layers.Conv2d(conv2d_18, 2, (1,1), W_init= W_init) 

    # logits tensor, often a step inbetween befor a softmax activation is applied (size im_size x im_size, 2)
    logits = output_image.outputs

    # the model
    model = output_image

    # a binary output map with size im_size x im_size can be found by applying the argmax operation
    # this operation yields 0 or 1 for every pixel position depending on whether the value in the first or second image is the largest
    binary_prediction = tf.math.argmax(logits, axis= 3, name= 'prediction')
        
    return binary_prediction, logits, output_image

The sigmoid focal cross-entropy is implemented making use of the source code: 
https://github.com/tensorflow/addons/blob/v0.10.0/tensorflow_addons/losses/focal_loss.py#L26-L85

In [ ]:
# define some different losses

# loss defined as in the original UNet paper
# The energy function is computed by a pixel-wise soft-max over the final feature map combined with the cross-entropy loss function
def loss_UNet(predicted_logits, real_annotations):
    # removes the last dimension of real_annotaitons (axis channel 1 has to be removed)
    real_annotations = real_annotations[:,:,:,0]
    
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits (logits= predicted_logits, labels= real_annotations)
    print(loss.shape)
    loss = tf.reduce_mean(loss) 
    print(loss.shape)
    
    # extra L2- regularization can be added to the loss
    # L2 = 0
    # for p in tl.layers.get_variables_with_name('/W', True, True):
    #   L2 += tf.contrib.layers.l2_regularizer(0.004)(p)
    # cost = loss + L2
    # print('Value of L2: {}'.format(L2))
        
    return loss


# here it is the softmax focal cross entropy
def loss_sfce(predicted_logits, real_annotations, alpha = 0.25, gamma = 2.0):
    '''
    This type of loss function tries to avoid data imbalance in image segmentation
    There are two parameters alpha and gamma, the default values are indicated
    gamma should always be greater than or equal to 0
    '''

    real_annotations = real_annotations[:,:,:,0]
    pred_prob = tf.nn.softmax(predicted_logits)[:,:,:,1]
    real_annotations = tf.cast(real_annotations, tf.float32)
    
    # classic binary cross_entropy is calculated
    ce = K.binary_crossentropy(real_annotations, pred_prob, from_logits= False)

    # binary cross-entropy is multiplied with two factors: alpha and modulating factor
    # convert the logits predictions into probabilities
    alpha_factor = 1.0
    modulating_factor = 1.0
    
    if alpha:
        alpha = tf.convert_to_tensor(alpha, dtype=K.floatx())
        alpha_factor = real_annotations * alpha + (1 - real_annotations) * (1 - alpha)


    p_t = (real_annotations * pred_prob) + ((1 - real_annotations) * (1 - pred_prob))
    if gamma:
        gamma = tf.convert_to_tensor(gamma, dtype=K.floatx())
        modulating_factor = tf.pow((1.0 - p_t), gamma)

    # compute the final loss and return
    loss = alpha_factor * modulating_factor * ce
    loss = tf.reduce_mean(loss)
    return loss


# Asymmetric similarity loss function, to balance recall and precision
# the larger beta, the more important the recall becomes relative to the precision
def loss_asl(predicted_logits, real_annotations, beta = 1):
    real_annotations = real_annotations[:,:,:,0]
    real_annotations = tf.cast(real_annotations, tf.float32)
    pred_prob = tf.nn.softmax(predicted_logits)[:,:,:,1]
    
    prod_pos = pred_prob * real_annotations
    sum_prod_pos = tf.reduce_sum(tf.reduce_sum(prod_pos, axis = 2), axis = 1)
    prod_neg_pred = (1-pred_prob) * real_annotations
    sum_prod_neg_pred = tf.reduce_sum(tf.reduce_sum(prod_neg_pred, axis = 2), axis = 1)
    prod_neg_real = (pred_prob) * (1-real_annotations)
    sum_prod_neg_real = tf.reduce_sum(tf.reduce_sum(prod_neg_real, axis = 2), axis = 1)

    beta = tf.convert_to_tensor(beta, dtype=K.floatx())

    num = (1+beta**2) * sum_prod_pos
    denom = (1+beta**2) *sum_prod_pos + beta**2 * sum_prod_neg_pred + sum_prod_neg_real

    loss = num/denom
    return tf.reduce_mean(loss)

In [ ]:
# define some optimizers and apply them to the network
def Adam_optimization(losses, lr = 0.001):
    '''This function defines the Adam optimizer that will be used and applies it to optimize the weights during training'''
    optimizer = tf.train.AdamOptimizer(learning_rate = lr)
    train_variables_list = tf.trainable_variables()
    gradients = optimizer.compute_gradients(losses, var_list= train_variables_list)
    train_optimization = optimizer.apply_gradients(gradients)
            
    return train_optimization

def SGD_optimization(losses, lr = 0.001):
    '''This function defines the Stochastic gradient descent optimizer that will be used and applies it to optimize the weights during trainin'''
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
    train_variables_list = tf.trainable_variables()
    gradients = optimizer.compute_gradients(losses, var_list= train_variables_list)
    train_optimization = optimizer.apply_gradients(gradients)

In [ ]:
# some training constants

print_freq = 1 # print frequency indicates how often results are printed out (amount of epochs)

In [ ]:
# # flags parameters, indicating in what mode the system is: training or testing and visualizing
# FLAGS = tf.flags.FLAGS
# tf.flags.DEFINE_string('mode', "train", "Mode train/ test/ visualize")

In [ ]:
def train_network(TrainImages, TrainAnnotations, TestImages, TestAnnotations, 
                  Drop_Prob = 0.1, Init_Filters = 64, batch_size = 3, loss_function = 'UNet_loss', optim = 'Adam', 
                  learning_rate = tf.Variable(1e-5, dtype=tf.float32), MAX_EPOCH = 10, SaveResults = True):
    '''
    This function trains the UNet on the indicated train data with corresponding annotations
    At the end the trained model is being saved
    '''

    # placeholders are created, variables to which data is assigned later on
    print('Create placeholders')
    image = tf.placeholder(tf.float32, [None,512,512, 3], name= 'image')
    annotation = tf.placeholder(tf.int32, shape=[None, 512, 512, 1], name= "annotation")

    # define the model that will be used for training and for testing
    print('Define the model for training')
    train_pred, train_logits, train_network = UNet(image, drop_prob = Drop_Prob, init_filters = Init_Filters, reuse = False, is_train = True)
    print('Define the model for testing')
    test_pred, test_logits, test_network = UNet(image, drop_prob = Drop_Prob, init_filters = Init_Filters, reuse = True, is_train = False)
    
    # define the output probability maps, losses and optimization for training and testing
    print('Define outputs, losses and optimization')
    # softmax activation creates a probability map which is the output
    train_positive_prob = tf.nn.softmax(train_logits)[:, :, :, 1]
    test_positive_prob = tf.nn.softmax(test_logits)[:, :, :, 1]
    # loss function
    if loss_function == 'UNet_loss':
        train_loss_op = loss_UNet(train_logits, annotation)
        test_loss_op= loss_UNet(test_logits, annotation)
    elif loss_function == 'Sfce_loss':
        train_loss_op = loss_sfce(train_logits, annotation, alpha = 0.25, gamma = 2.0)
        test_loss_op= loss_sfce(test_logits, annotation, alpha = 0.25, gamma = 2.0)
    elif loss_function == 'Asl_loss':
        train_loss_op = loss_asl(train_logits, annotation)
        test_loss_op= loss_asl(test_logits, annotation)
    # optimization
    if optim == 'Adam':
        train_op = Adam_optimization(train_loss_op, learning_rate)
    elif optim == 'sgd':
        train_op = SGD_optimization(train_loss_op, learning_rate)
        
    # learning_rate decay at higher amounts of epochs 
    lr_assign_op = tf.assign(learning_rate, learning_rate / 10)
    
    # a session should be started in which all intermediate results are stored (also weights of the network for example)
    sess = tf.Session()
    # saving the train and test results and the trained model
    if SaveResults:
        # creating summary which stores the results that can be visualised with tensorboard
        print("Setting up summary...")
        # test_summary_op = tf.summary.merge_all()
        # creating saver, used to save the trained model at the end
        print("Setting up Saver...")
        saver = tf.train.Saver(max_to_keep=2)
        summary_writer = tf.summary.FileWriter(log_dir_tens, sess.graph)

        
    tl.layers.initialize_global_variables(sess)
    sess.run(tf.global_variables_initializer())
    
    # define the train and test batches that can be fed into the network
    train_batch_data, test_batch_data = LoadBatchData(TrainImages, TrainAnnotations, TestImages, TestAnnotations)

    # defining the numer of steps per epoch, based on the batchsize
    train_nbr = TrainImages.shape[0]
    test_nbr = TestImages.shape[0]
    step_every_epoch = int(train_nbr/batch_size)
    test_every_epoch = int(test_nbr/batch_size)
    
    # go over all epochs of training
    for epo in range(1,MAX_EPOCH+1):
        
        # time is defined at the start of every epoch
        start_time = time.time()
        # resetting of all variables
        train_loss, test_loss, train_aupr, test_aupr, train_auc, test_auc= 0, 0, 0, 0, 0, 0
            
        # go over all batches in one epoch
        for s in range(step_every_epoch):
        
            # define the next batch to train the network
            train_images_batch, train_annotations_batch = train_batch_data.next_batch(batch_size)
            feed_dict = {image: train_images_batch, annotation: train_annotations_batch}
            # train the network and define the output of the network for this batch of images
            train_pos_prob, train_err, _ = sess.run([train_positive_prob, train_loss_op, train_op], feed_dict=feed_dict)

            # auc and aupr score are calculated for this batch
            temp_train_annotations = np.reshape(train_annotations_batch,-1)
            temp_tra_positive_prob = np.reshape(train_pos_prob,-1)
            train_sauc = ROC(temp_tra_positive_prob, temp_train_annotations, plot = False)
            train_saupr = PRC(temp_tra_positive_prob, temp_train_annotations, plot = False)

            # define the total loss, auc and aupr over all batches in 1 epoch
            train_loss += train_err
            train_auc += train_sauc
            train_aupr += train_saupr
            
        # after every epoch, check whether results should be printed out and network should be tested
        # print out after epoch 1 and then every print_freq
        if epo % print_freq == 0 or epo == 1 or epo == (MAX_EPOCH):
        
            # average out loss, auc and aupr for the train set over all batches
            train_loss = train_loss/step_every_epoch
            train_auc = train_auc/step_every_epoch
            train_aupr = train_aupr/step_every_epoch

            # print out the training results
            print('epoch {} took {}s'.format(epo, time.time() - start_time))
            print('   train loss: {}'.format(train_loss))
            print('   train auc: {}'.format(train_auc))
            print('   train aupr: {}'.format(train_aupr))

            if SaveResults:      
                # save these values to visualize them later with tensorboard
                train_summary = tf.Summary(value=[tf.Summary.Value(tag="train_loss", simple_value=train_loss), tf.Summary.Value(tag="train_auc", simple_value=train_auc),
                                                tf.Summary.Value(tag="train_aupr", simple_value=train_aupr)])
                summary_writer.add_summary(train_summary, epo)

            # testing of the network
            # test data is also subdivided in batches, go over all batches
            for test_s in range(test_every_epoch):

                # get the next batch of test data
                test_images_batch, test_annotations_batch = test_batch_data.next_batch(batch_size)
                # apply the network to the test images and define the output of the network and the loss
                feed_dict= {image:test_images_batch, annotation:test_annotations_batch}
                test_pos_prob, test_err = sess.run([test_positive_prob, test_loss_op], feed_dict= feed_dict)

                # compute auc and aupr score for test set
                temp_test_annotations = np.reshape(test_annotations_batch,-1)
                temp_test_positive_prob = np.reshape(test_pos_prob,-1)
                test_sauc = ROC(temp_test_positive_prob, temp_test_annotations, plot = False)
                test_saupr= PRC(temp_test_positive_prob, temp_test_annotations, plot = False)

                # calculate total loss, auc and aupr for the test set over all batches
                test_loss += test_err
                test_auc += test_sauc
                test_aupr += test_saupr

            # average loss, auc and aupr for the test set over all batches
            test_loss = test_loss/test_every_epoch
            test_auc = test_auc/test_every_epoch
            test_aupr = test_aupr/test_every_epoch

            # print out the test results
            print('   test loss: {}'.format(test_loss))
            print('   test auc: {}'.format(test_auc))
            print('   test aupr: {}'.format(test_aupr))

            if SaveResults:       
                # save these values to visualize them later with tensorboard
                # print('1')
                test_summary = tf.Summary(value=[tf.Summary.Value(tag="test_loss", simple_value=test_loss), tf.Summary.Value(tag="test_auc", simple_value=test_auc), 
                                                tf.Summary.Value(tag="test_aupr", simple_value=test_aupr)])
                # print('2')
                summary_writer.add_summary(test_summary, epo)

                # # visualize the test result(only visualize the last batchsize of this epoch)
                # print('3')
                # feed_dict= {image:test_images_batch, annotation:test_annotations_batch}
                # print('4')
                # print(type(test_summary_op))
                # summary_str = sess.run(test_summary_op, feed_dict = feed_dict)
                # print('5')
                # summary_writer.add_summary(summary_str, epo)
                # print('6')   

                # tensorboard flush
                summary_writer.flush() # the summary is written at this moment
                # sys.stdout.flush()
            
        # at specific amounts of epochs, the learning rate should become smaller to work more precisely
        if epo == int(MAX_EPOCH*2/3) or epo == int(MAX_EPOCH/2): 
            sess.run(lr_assign_op)
            
        # eventual saving of the fully trained model
        if SaveResults: 
            if epo % (MAX_EPOCH) == 0:
                saver.save(sess, log_dir_model + "model.ckpt", epo)
                print('epoch {}, the model has been saved successfully'.format(epo))
                # force the system to write now
                # sys.stdout.flush()

    return test_loss, test_auc, test_aupr   

In [ ]:
train_network(train_images, train_annotations, test_images, test_annotations, MAX_EPOCH = 10, loss_function = 'UNet_loss')
# TrainImages, TrainAnnotations, TestImages, TestAnnotations, 
#                   Drop_Prob = 0.1, Init_Filters = 64, batch_size = 3, loss_function = 'UNet_loss', optim = 'Adam', 
#                   learning_rate = tf.Variable(1e-3, dtype=tf.float32), MAX_EPOCH = 10, SaveResults = True

Create placeholders
Define the model for training
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


[TL] InputLayer  input: (?, 512, 512, 3)
[TL] Conv2d conv2d: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu


[TL] Conv2d conv2d: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d maxpool_1: filter_size: (2, 2) strides: (2, 2) padding: SAME
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Please use `layer.__call__` method instead.
[TL] DropoutLayer dropout_layer: keep: 0.900000 is_fix: True
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to

KeyboardInterrupt: ignored

In [ ]:
# visualize the training results with tensorboard
%tensorboard --logdir {log_dir_tens}  --host localhost

In [ ]:
def Reload_and_Predict(TestImages, TestAnnotations, Drop_Prob = 0.1, Init_Filters = 64, log_dir_trained_model = log_dir_model, test_batch = 6):
    '''This function reloads a trained model and predicts the annotations of images'''
    TestAnnotations = np.expand_dims(TestAnnotations, axis = 3)
    TestAnnotations = np.where(TestAnnotations> 0, 1, 0)
    
    # placeholders are created, variables to which data is assigned later on
    print('Create placeholders')
    image = tf.placeholder(tf.float32, [None,512,512, 3], name= 'image')
    annotation = tf.placeholder(tf.int32, shape=[None, 512, 512, 1], name= "annotation")

    # Define the model for testing
    print('Define the model for testing')
    test_pred, test_logits, test_network = UNet(image, drop_prob = Drop_Prob, init_filters = Init_Filters, reuse = True, is_train = False)
        
    # softmax activation of the output to end up with probability maps
    print('Define the probability outputs')
    test_positive_prob = tf.nn.softmax(test_logits)[:, :, :, 1]
        
    # start up a session in which the pretrained model is reloaded
    sess = tf.Session()    
    tl.layers.initialize_global_variables(sess)
    sess.run(tf.global_variables_initializer())
    
    print('Restoring model...')
    new_saver = tf.train.import_meta_graph(log_dir_trained_model + 'model.ckpt-10.meta')
    ckpt = tf.train.get_checkpoint_state(log_dir_trained_model)
    new_saver.restore(sess, ckpt.model_checkpoint_path)
    print('Model restored...')
        
    # predict the annotations for the test images
    print('Predicting the test cases...')

    # the test samples have to go through in batches as otherwise the memmory crashes
    # number of batches
    predicted_prob_maps = []
    n_batches = int(TestImages.shape[0] / test_batch)

    for i in range(n_batches):
        start = i*test_batch
        end = start + test_batch
        feed_dict= {image: TestImages[start:end]}
        test_pos_prob= sess.run([test_positive_prob], feed_dict= feed_dict)
        test_pos_prob = np.array(test_pos_prob)

        if i == 0:
            predicted_prob_maps = test_pos_prob[0,:,:,:]
        else:
            predicted_prob_maps = np.vstack((predicted_prob_maps, test_pos_prob[0,:,:,:]))

    print('Test data predicted')
    
    # test_positive_prob gives the output probability map that can be used as an input to the test function
    return predicted_prob_maps

In [ ]:
predicted_annot = Reload_and_Predict(test_images, test_annotations, Drop_Prob = 0.1, Init_Filters = 64, log_dir_trained_model = log_dir_model)

NameError: ignored

Beneith some functions are indicated that can be used to evaluate the trained model. They are all brought together in the Evaluate funtion.

In [ ]:
def ROC(pred_probs, true_probs, plot = True):
    '''
    This function calculates the ROC-AUC value
    and it also calculates and visualizes the ROC-curve (if plot is true)
    '''
    
    # calculate and print out ROC-AUC value
    ROC_AUC = roc_auc_score(true_probs, pred_probs)
    
    # calculate and plot the ROC-curve
    if plot:
        FPRate, TPRate, Thresh = roc_curve(true_probs, pred_probs)
        plt.figure()
        plt.plot(FPRate, TPRate)
        plt.title('ROC curve')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.show()

    return ROC_AUC

In [ ]:
def PRC(pred_probs, true_probs, plot = True):
    '''
    Calculate the aupr value = the area under the precision-recall curve
    and plot the precision-recall curve (if plot is true)
    '''
    
    # calculate the precision-recall curve
    Precision, Recall, Thresh = precision_recall_curve(true_probs, pred_probs)
    Precision = np.fliplr([Precision])[0]  # so the array is increasing (you won't get negative AUC)
    Recall = np.fliplr([Recall])[0]  # so the array is increasing (you won't get negative AUC)
    AUPR = np.trapz(Precision, Recall)
      
    
    # plot the precision-recall curve
    if plot:
        plt.figure()
        plt.plot(Precision, Recall)
        plt.title('Precision-Recall curve')
        plt.xlabel('Precision')
        plt.ylabel('Recall')
        plt.show()
    
    return AUPR

In [ ]:
def ConfusionMatrix(pred_probs, true_probs, threshold_confusion = 0.5):
    '''
    Calculate and print out the confusion matrix
    The values for the confusion matrix are also returned
    A standard threhsold of 0.5 is used to calculate the confusion matrix
    '''
    # print out the threshold that is used for the confusion matrix
    print("Confusion matrix with a used threshold of {} for the positive class:".format(threshold_confusion))
    
    # turn the predicted probability maps into binary outputs according to the threshold
    thresh_pred_probs = np.empty((pred_probs.shape[0]))
    for i in range(pred_probs.shape[0]):
        if pred_probs[i] >= threshold_confusion:
            thresh_pred_probs[i] = 1
        else:
            thresh_pred_probs[i] = 0
    
    # calculate the confusion-matrix and print it out
    TN, FP, FN, TP = confusion_matrix(true_probs, thresh_pred_probs).ravel()
    print('Amount of true positives: {}'.format(TP))
    print('Amount of false positives: {}'.format(FP))
    print('Amount of true negatives: {}'.format(TN))
    print('Amount of false negatives: {}'.format(FN))
    
    return TN, FP, FN, TP

In [ ]:
def Metrics(tn, fp, fn, tp):
    '''Print out some metrics like the accuracy, specificity, sensitivity and precision'''
    
    print('Some metrics:')
    
    # print out accuracy
    accuracy = 0
    if (tn+fp+fn+tp) != 0:
        accuracy = (tp+tn)/(tn+fp+fn+tp)
    print('Accuracy: {}'.format(accuracy))
    
    # print out specificity
    specificity = 0
    if (tn+fp) != 0:
        specificity = tn/(tn+fp)
    print('Specificity: {}'.format(specificity))
    
    # print out sensitivity
    sensitivity = 0
    if (tp+fn) != 0:
        sensitivity = tp/(tp+fn)
    print('Sensitivity: {}'.format(sensitivity))
    
    # print out precision
    precision = 0
    if (tp+fp) != 0:
        precision = tp/(tp+fp)
    print('Precision: {}'.format(precision))


In [ ]:
# evaluation of the model
def Evaluate(pred_prob_maps, true_prob_maps):
    '''
    Evaluation of the network, based on comparisson between the true and predicted outputs
    '''
    print('Evaluating the model...')
    
    # creat a 1D numpy array with the predicted and true outputs
    pred_prob_maps = pred_prob_maps.reshape(-1)
    true_prob_maps = true_prob_maps.reshape(-1)

    # print out ROC-AUC value and plot the ROC-curve
    ROC_AUC = ROC(pred_prob_maps, true_prob_maps)
    print('Area under the ROC curve: {}'.format(ROC_AUC))

    # print out area under the precision-recall curve, AUPR value
    AUPR = PRC(pred_prob_maps, true_prob_maps)
    print('Area under Precision-Recall curve: {}'.format(AUPR))
    
    # print out confusion matrix
    TN, FP, FN, TP = ConfusionMatrix(pred_prob_maps, true_prob_maps)
    
    # print out some metrics
    Metrics(TN, FP, FN, TP)

In [ ]:
Evaluate(predicted_annot, test_annotations)

The functions beneith can be used to perform a gridsearch over different combinations of parameters to define the parametercombination that gives the best results in terms of performance and training time.

In [ ]:
# gridsearch: look for the most optimal hyperparameters, making use of cross-validation

# for this a separate validation set has to be defined
# three fold cross-validation is performed in this case
def ThreeFoldSplit(n_split):
    '''
    This function calculates a three-fold split of the train_images and train_annotations
    Depending on n_split another train and validation set is defined
    '''
    
    a = train_images.shape[0]
    b = int(a/3)
    
    val_images = train_images[n_split*b:(n_split+1)*b]
    val_annotations = train_annotations[n_split*b:(n_split+1)*b]
    
    # split the train set in to three parts
    if n_split == 0:
        tr_images = train_images[b:]
        tr_annotations = train_annotations[b:]
        
    if n_split == 1:
        tr_images = np.vstack((train_images[0:b], train_images[2*b:]))
        tr_annotations = np.vstack((train_annotations[0:b], train_annotations[2*b:]))
        
    if n_split == 2:
        tr_images = train_images[0:2*b]
        tr_annotations = train_annotations[0:2*b]
    
    return tr_images, tr_annotations, val_images, val_annotations

In [ ]:
# function to choose the best hyperparameter combination based on model performance in terms of the ROC AUC score
def Hyperparam_Optimization(lr_list, Batch_Size_list, dropout_list, loss_func_list, optim_list, 
                            init_filters_list, epochs_list, filename):
    n = 0
    file = open('/content/drive/My Drive/Stage_ENT_Studios/Unet/GridSearchResults/'+filename+'.txt','w') 
    for LearnRate in lr_list:
        for BatchSize in Batch_Size_list:
            for Dropout in dropout_list:
                for LossFunc in loss_func_list:
                    for Optim in optim_list:
                        for InitFilters in init_filters_list:
                            for Epoch in epochs_list:
                                
                                n +=1
                                print('Evaluating parameter combination {} ...'.format(n))
                                    
                                AUC_list = []
                                AUPR_list = []
                                Loss_list = []
                                time_list = []
                                # 3-fold cross-validation is used to evaluate the model 
                                # for a certain param combination 
                                for fold in range(3):
                                        
                                    tr_images, tr_annotations, val_images, val_annotations = ThreeFoldSplit(fold)
                                    
                                    # fitting the model to the data for a certain fold and defining the auc, aupr and loss
                                    start_time = time.time()
                                    
                                    val_loss, val_auc, val_aupr = train_network(tr_images, tr_annotations, val_images, val_annotations, Dropout, InitFilters, BatchSize, LossFunc, Optim, 
                                                  LearnRate, Epoch, SaveResults = False)
                                    
                                    end_time = time.time()
                        
                                    # the ROC auc value and the PR auc value can be compared to each other
                                    # for different param combinations
                                        
                                    # creat a 1D numpy array with the predicted and true outputs   
                                    AUC_list.append(val_auc)
                                    AUPR_list.append(val_aupr)
                                    Loss_list.append(val_loss) 
                                    time_list.append(end_time-start_time) 
                                    
                                # calculate the mean and standard-dev of the AUC, AUPR, loss and time for all three folds
                                mean_AUC = np.mean(np.array(AUC_list), axis = 0)
                                std_AUC = np.std(np.array(AUC_list), axis = 0)
                                mean_AUPR = np.mean(np.array(AUPR_list), axis = 0)
                                std_AUPR = np.std(np.array(AUPR_list), axis = 0)
                                mean_Loss = np.mean(np.array(Loss_list), axis = 0)
                                std_Loss = np.std(np.array(Loss_list), axis = 0)
                                mean_time = np.mean(time_list, axis = 0)
                                    
                                # print out these values together with the training time for this parameter set
                                print('Hyperparameter combination:')
                                print('learning rate {}, batch size {}, dropout {}, loss function {},optimization {}, initial amount of filters {} and amount of epochs {}'.format(LearnRate, BatchSize, Dropout, LossFunc, Optim, InitFilters, Epoch))
                                print('Results:')
                                print('AUC mean: {} and standard deviation: {}'.format(mean_AUC, std_AUC))
                                print('AUPR mean: {} and standard deviation: {}'.format(mean_AUPR, std_AUPR)) 
                                print('Loss mean: {} and standard deviation: {}'.format(mean_Loss, std_Loss)) 
                                print('training time: {}'.format(mean_time))

                                # save all values in a txt file
                                file.write('Hyperparameter combination: \n')
                                file.write('learning rate {}, batch size {}, dropout {}, loss function {},optimization {}, initial amount of filters {}, amount of epochs {} \n'.format(LearnRate, BatchSize, Dropout, LossFunc, Optim, InitFilters, Epoch))
                                file.write('AUC mean: {} and standard deviation: {} \n'.format(mean_AUC, std_AUC))
                                file.write('AUPR mean: {} and standard deviation: {} \n'.format(mean_AUPR, std_AUPR))
                                file.write('Loss mean: {} and standard deviation: {} \n'.format(mean_Loss, std_Loss))
                                file.write('training time: {} \n'.format(mean_time))
                                file.write(' \n')

    file.close() 
                                                              

In [ ]:
# different options that have to be tried
# learn rat
LR_list = [tf.Variable(1e-4, dtype=tf.float32), tf.Variable(1e-5, dtype=tf.float32), tf.Variable(1e-6, dtype=tf.float32)]
# batch size
BS_list = [3,6]
# drop probab
DP_list = [0.1,0.3]
# loss function
LF_list = ['UNet_loss'] # Sfce loss can not really be used in this case
# optimisation function
OF_list = ['Adam', 'sgd']
# initial amount of filters
IF_list = [64, 32, 16]
# amount of epochs
EP_list = [10, 50]
# alpha and gamma, parameters of the focal loss function

Hyperparam_Optimization(lr_list = LR_list, Batch_Size_list = BS_list, dropout_list = DP_list, 
                        loss_func_list = LF_list, optim_list = OF_list, init_filters_list = IF_list, 
                        epochs_list = EP_list, filename = 'GridSearch_UNet_Softmax')

In [ ]:
# to visualize some predictions
def PlotImage(image_array, database = '', image_name = '', save = False, savepath = ''):
    '''Visualize (and save if desired) an image represented by a numpy array'''
    
    plt.figure()
    plt.title(database+': '+image_name)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.imshow(image_array, cmap=plt.cm.Greys_r)
    
    if save == True:
        plt.savefig(savepath+database+'_'+image_name+'.png')
    
    return 

In [ ]:
annotation = predicted_annot[0]
positives = np.where(annotation > 0.5)
bin_annotation = np.zeros(annotation.shape)
bin_annotation[positives] = 1
PlotImage(annotation)
PlotImage(bin_annotation)

To kill an active tensorboard session:
Go to the anaconda prompt and type:

taskkill /im tensorboard.exe /f

del /q %TMP%\.tensorboard-info\*


or in the windows command prompt:

taskkill /IM "tensorboard.exe" /F